Explore, segment, and cluster the neighborhoods in the city of Toronto

First import the libraries we need

In [2]:
#Un-comment if needed
#!pip install beautifulsoup4
#!pip install lxml
#!pip install requests

#Import libraries

from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# import k-means from clustering stage
#from sklearn.cluster import KMeans


import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.37 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.26 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.22 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.56 MB/s
vincent-0.4.4- 100% |###################

Now download the data for Toronto neighborhoods

In [3]:
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


Use BeautifulSoup to extract data from downloaded data

In [4]:
with open('toronto_data.html') as html_file:
    tsoup = BeautifulSoup(html_file,'lxml')
#print(toronto_soup.prettify())

In [5]:
datacols = []
for th in tsoup.find_all('th'):
    datacols.append(th.text.rstrip())
    


In [6]:
alldata = []
datarows = []
for td in tsoup.find_all('td'):
    alldata.append(td.text.rstrip())

rowlen = len(datacols)-1
numrows = int(len(alldata)/rowlen)

for rnum in range(0,numrows):
    row=[]
    for i in range(0,rowlen):
        row.append(alldata[i+rnum*rowlen])
    if (len(row[0])==3):
        datarows.append(row)



Now create Dataframe from extracted data and remove rows with Borough = Not assigned

In [7]:
df = pd.DataFrame(columns=datacols[0:3])
df2 = pd.DataFrame(datarows,columns=datacols[0:3])
dfToronto=df.append(df2)

dfToronto['Borough'] = dfToronto['Borough'].replace('Not assigned',np.nan)

dfToronto.dropna(axis=0, inplace=True)

dfToronto.head(10)



,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Now find which rows have multiple neighborhoods for same postcode

In [8]:
dup=dfToronto.duplicated(subset='Postcode')
ddup = dup.to_dict()
dupt = []
for key in ddup:
    if ddup[key]:
        dupt.append(key)
    

Now get the list of postcodes which have more than one neighborhood and remove duplicates.

In [9]:
dbor = []
for index in dupt:
    dbor.append(dfToronto.loc[(index,'Postcode')])
dbor1 = list(set(dbor))
print(dbor1)


['M6N', 'M5P', 'M1E', 'M8V', 'M5X', 'M8W', 'M9R', 'M2M', 'M1C', 'M8Z', 'M3H', 'M6J', 'M5S', 'M9B', 'M4B', 'M3C', 'M4L', 'M4T', 'M4V', 'M1R', 'M4K', 'M1W', 'M6A', 'M6L', 'M9V', 'M1V', 'M6K', 'M6P', 'M8X', 'M9M', 'M5T', 'M5B', 'M1T', 'M5V', 'M1B', 'M2J', 'M5A', 'M6H', 'M1L', 'M8Y', 'M5M', 'M6S', 'M5R', 'M6R', 'M6M', 'M5L', 'M4X', 'M2L', 'M1N', 'M1M', 'M1P', 'M9C', 'M5H', 'M5J', 'M5K', 'M1K', 'M3K', 'M3J']



Create dictionary of dataframe rows which have multiple neighborhoods

In [10]:
mn = {}
for pc in dbor1:
    nlist = []
    for index,row in dfToronto.iterrows():
        if row["Postcode"] == pc:
            nlist.append(row["Neighbourhood"])
    mn[pc] = nlist
    
        

Drop duplicates from dataframe, based on postcode

In [11]:
dfT1 = dfToronto
dfT1.drop_duplicates(subset='Postcode',inplace=True)
dfT1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
8,M7A,Queen's Park,Not assigned


Now replace the neighborhoods with lists from dictionary and assign Borough to Not Assigned neighborhoods

In [12]:
for pc in dbor1:
    nlist = mn[pc]
    nstr = ','.join(nlist)
    for index,row in dfT1.iterrows():
        if row["Postcode"] == pc:
            row["Neighbourhood"] = nstr
        if row["Neighbourhood"] == "Not assigned":
            row["Neighbourhood"] = row["Borough"]

dfT1.head(10)       


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


Reset the dataframe index 

In [13]:
dfT1.reset_index(drop=True,inplace=True)
dfT1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Just to be picky change Neighbourhood to conform with assignment instructions  

In [14]:
dfT1.rename(columns={'Neighbourhood': 'Neighborhood'},inplace=True)

In [15]:
dfT1

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [16]:
dfT1.shape

(103, 3)

Now get location data

In [17]:
path='https://cocl.us/Geospatial_data'
dfLoc = pd.read_csv(path)
dfLoc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add the location columns to neighborhood dataframe

In [18]:
dfT2 = pd.concat([dfT1, dfLoc],axis=1).drop(['Postal Code'], axis=1)

for index,row in dfT2.iterrows():
    pc = row["Postcode"]
    lat = dfLoc.loc[(dfLoc['Postal Code'] == pc),"Latitude"]
    long = dfLoc.loc[(dfLoc['Postal Code'] == pc),"Longitude"]
    dfT2.at[index, 'Latitude'] = lat
    dfT2.at[index, 'Longitude'] = long
    
   
        

dfT2.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Select the rows with Toronto in name of Borough

In [19]:
#[ 'Downtown Toronto', 'Central Toronto',  'East Toronto']]
dfDown = dfT2[dfT2.Borough == 'Downtown Toronto']
dfCentral = dfT2[dfT2.Borough == 'Central Toronto']
dfEast = dfT2[dfT2.Borough == 'East Toronto']

dfToro = pd.concat([dfDown, dfCentral, dfEast])

dfToro


,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752
42,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817


Create a map of Toronto with neighborhoods marked

In [38]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToro['Latitude'], dfToro['Longitude'], dfToro['Borough'], dfToro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Now get location data from Foursquare for postcode M5S.

In [20]:
CLIENT_ID = 'GBLG342Y3PHJQUSLATKNBBHOJTHG4OUMABBTAIE32YQZDYXG' # your Foursquare ID
CLIENT_SECRET = 'HUKODZGFDWNJLZQSGBFZVPWKDBWEOEQE53WCLR4HOZ3QFI1K' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 500

In [39]:
index=dfToro.index[dfToro['Postcode'] == 'M5S'][0]
latitude = dfToro.at[index,'Latitude']
longitude = dfToro.at[index,'Longitude']
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
results = requests.get(url).json()
results
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-5362c366498e602fbe1db395-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d111941735',
    'name': 'Japanese Restaurant',
    'pluralName': 'Japanese Restaurants',
    'primary': True,
    'shortName': 'Japanese'}],
  'id': '5362c366498e602fbe1db395',
  'location': {'address': '81 Harbord St.',
   'cc': 'CA',
   'city': 'Toronto',
   'country': 'Canada',
   'distance': 255,
   'formattedAddress': ['81 Harbord St.', 'Toronto ON M5S 1G4', 'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.66283719650635,
     'lng': -79.40321739973975}],
   'lat': 43.66283719650635,
   'lng': -79.40321739973975,
   'neighborhood': 'Harbord Village, Toronto, ON',
   'postalCode': 'M5S 1G4',
   'state': 'ON'},
  'name': 'Y

Now explore the Harbord neighborhood.

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
venues = results['response']['groups'][0]['items']

    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Almond Butterfly,Bakery,43.662836,-79.403365
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746


In [42]:
#get the number of venues returned
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.
